In [122]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Costruzione del dataset

In [123]:
credit = pd.read_csv("C:/Users/Vittorio D'Onofrio/Downloads/credit_record.csv" , sep=',' )

In [124]:
application = pd.read_csv ( "C:/Users/Vittorio D'Onofrio/Downloads/application_record.csv" , sep=',' )

L'idea è quella di impostare un problema di classificazione binario, in cui la colonna target vale 1 se il cliente è un cattivo pagatore e 0 altrimenti.\
La tabella credit contiene lo storico dei pagamenti dei clienti a cui è stata data la carta ed è da qui che possiamo dedurre (o meglio ipotizzare) che un determinato id sia o meno un cattivo pagatore.

In [125]:
credit.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


Il criterio scelto per vedere se un cliente è un cattivo pagatore o meno è il seguente: se la media dei suoi ritardi nei pagamenti supera 59 giorni allora è un cattivo pagatore.\
La media sembra essere la statistica migliore per questo problema perchè è sensibile agli outliers e quindi anche solo un ritardo di più di 150 giorni può influire nella valutazione del cliente e questo sembra avere senso.\
In generale non c'è un modo univoco per identificare un cattivo pagatore ed il criterio dovrebbe essere scelto anche in accordo con le linee guida dell'istituto per cui si svolge questo progetto.\
Infatti, la banca potrebbe decidere di prendersi meno rischio e classificare come cattivo pagatore ad esempio uno che ha una media di ritardi nei pagamenti anche al di sotto di due mesi, oppure di classificare a priori come cattivo pagatore uno che ha avuto almeno una volta più di 150 giorni di ritardo.\
Viceversa, la banca può decidere di assumere più rischio e alzare la soglia della media per la classificazione: si potrebbe ad esempio stabilire che un cliente è un cattivo pagatore quando la media dei suoi ritardi nei pagamenti supera ad esempio 119 giorni.

Con la cella seguente creo un dataframe "credit_new" in cui un cliente è classificato come cattivo pagatore quando la media dei ritardi nei pagamenti supera 59 giorni.

In [126]:
credit_new = pd.DataFrame(columns=['ID', 'IS_BAD_PAYER'])

for id in set(credit['ID']):
    
    delay_list = []
    df = credit[credit['ID'] == id]
    
    for index, row in df.iterrows():
        selected = row['STATUS']
        if selected == 'X' or selected =='C':
            df.at[index,'STATUS'] = 0
        else:
            df.at[index,'STATUS'] = int(selected) + 1
        
        num_value = df.at[index,'STATUS']
    
        if num_value > 0:
            delay_list.append(num_value)
        
    if len(delay_list)==0:
        flag_bad_payer = 0
    else:
        mean_delay = sum(delay_list)/len(delay_list)
        if mean_delay > 2:
            flag_bad_payer = 1
        else:
            flag_bad_payer = 0
        
    new_row = {'ID': id , 'IS_BAD_PAYER': flag_bad_payer }   
    credit_new = pd.concat([credit_new, pd.DataFrame([new_row])], ignore_index=True)

In [127]:
credit_new[credit_new['IS_BAD_PAYER']==1]

,ID,IS_BAD_PAYER
786,5113621,1
1014,5113920,1
1025,5113933,1
1032,5113944,1
1268,5114236,1
...,...,...
44948,5105045,1
44956,5105056,1
45164,5105325,1
45554,5105788,1


In [128]:
credit_new['IS_BAD_PAYER'].value_counts()

0    45774
1      211
Name: IS_BAD_PAYER, dtype: int64

Creo il dataframe su cui addestrare il modello attraverso una inner join tra la tabella delle applicazioni e la credit_new appena creata.

In [129]:
data = pd.merge(application, credit_new, on='ID', how='inner')

Per comodità si può esportare il dataset appena creato in csv.

In [130]:
data.to_csv('data.csv', index=False)

In [131]:
data = pd.read_csv("C:/Users/Vittorio D'Onofrio/Downloads/data.csv", sep=';')

In [132]:
data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,IS_BAD_PAYER
0,5008804,M,Y,Y,0,4275000,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,20,0
1,5008805,M,Y,Y,0,4275000,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,20,0
2,5008806,M,Y,Y,0,1125000,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,20,0
3,5008808,F,N,Y,0,2700000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,10,0
4,5008809,F,N,Y,0,2700000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,10,0


In [133]:
data.shape

(36457, 19)

In [134]:
data['IS_BAD_PAYER'].value_counts()

0    36261
1      196
Name: IS_BAD_PAYER, dtype: int64

Il problema di classificazione è chiaramente sbilanciato, ha senso visto che ci si aspetta che tra i clienti di una banca ci siano pochi cattivi pagatori e moltissimi buoni pagatori.

In questo problema di classificazione sbilanciato quindi ci interessa massimizzare la recall.\
Infatti, una recall alta significa che c'è un basso numero di falsi negativi, cioè un basso numero di cattivi pagatori che il modello ha visto come buoni pagatori. In questo modo la banca si assicura di dare la carta solo a buoni pagatori.\
Inevitabilmente, massimizzando la recall si avrà una precision minore, cioè si avrà un alto numero di falsi positivi cioè di clienti che il modello ha visto come cattivi pagatori ma che in realtà sono buoni pagatori, quindi la banca su questo perderà dei potenziali buoni clienti ma si salvaguarderà dal rischio di cattivi pagatori.

# Preprocessing

In [135]:
data = pd.read_csv("C:/Users/Vittorio D'Onofrio/Downloads/data.csv", sep=';')

Intuitivamente, le variabili che indicano se un'osservazione ha o meno cellulare o una mail non sono indicative del fatto che quel cliente possa essere o meno un buon pagatore, quindi le eliminiamo dal dataset.

In [136]:
data = data.drop(['FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL'], axis=1)

In [137]:
data.isna().sum()

ID                         0
CODE_GENDER                0
FLAG_OWN_CAR               0
FLAG_OWN_REALTY            0
CNT_CHILDREN               0
AMT_INCOME_TOTAL           0
NAME_INCOME_TYPE           0
NAME_EDUCATION_TYPE        0
NAME_FAMILY_STATUS         0
NAME_HOUSING_TYPE          0
DAYS_BIRTH                 0
DAYS_EMPLOYED              0
OCCUPATION_TYPE        11323
CNT_FAM_MEMBERS            0
IS_BAD_PAYER               0
dtype: int64

Per il momento riempiamo i valori NaN della feature "OCCUPATION_TYPE" con una scritta indicante che manca il valore, ci occupiamo in seguito dell'encoding di questa feature.

In [138]:
data['OCCUPATION_TYPE'] = data['OCCUPATION_TYPE'].fillna("valore assente")

Procediamo con una one-hot encoding per la variabile indicante il sesso.

In [139]:
data = pd.get_dummies( data, columns=['CODE_GENDER' ] )

Le features "FLAG_OWN_CAR" e "FLAG_OWN_REALTY" possono essere trasformate in numeriche con una codifica binaria mentre per la variabile "NAME_EDUCATION_TYPE" si può usare l'ordinal encoding.

In [140]:
car_realty_mapping={
    "Y":1,
    "N":0
}

education_mapping = {
    "Academic degree": 5,
    "Higher education": 4,
    "Incomplete higher": 3,
    "Lower secondary": 2,
    "Secondary / secondary special": 1
}


data["FLAG_OWN_CAR"]=data["FLAG_OWN_CAR"].map(car_realty_mapping)
data["FLAG_OWN_REALTY"]=data["FLAG_OWN_REALTY"].map(car_realty_mapping)
data["NAME_EDUCATION_TYPE"]=data["NAME_EDUCATION_TYPE"].map(education_mapping)

In [141]:
data.head()

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,IS_BAD_PAYER,CODE_GENDER_F,CODE_GENDER_M
0,5008804,1,1,0,4275000,Working,4,Civil marriage,Rented apartment,-12005,-4542,valore assente,20,0,0,1
1,5008805,1,1,0,4275000,Working,4,Civil marriage,Rented apartment,-12005,-4542,valore assente,20,0,0,1
2,5008806,1,1,0,1125000,Working,1,Married,House / apartment,-21474,-1134,Security staff,20,0,0,1
3,5008808,0,1,0,2700000,Commercial associate,1,Single / not married,House / apartment,-19110,-3051,Sales staff,10,0,1,0
4,5008809,0,1,0,2700000,Commercial associate,1,Single / not married,House / apartment,-19110,-3051,Sales staff,10,0,1,0


In [142]:
data['DAYS_EMPLOYED'].value_counts(ascending=True)

-11272        1
-4507         1
-7698         1
-4686         1
-2160         1
           ... 
-2087        61
-200         63
-1539        64
-401         78
 365243    6135
Name: DAYS_EMPLOYED, Length: 3640, dtype: int64

Abbiamo un solo valore positivo molto alto per la variabile 'DAYS_EMPLOYED', questo valore corrisponde ad un unico "NAME_INCOME_TYPE" cioè ai pensionati. Questo valore è chiaramente un outlier e potrebbe influenzare la capacità previsionale di un modello. Cambiamolo quindi con 0.

In [143]:
data.loc[data['DAYS_EMPLOYED'] > 0, 'DAYS_EMPLOYED'] = 0

Controlliamo se nelle features numeriche ci sono delle correlazioni lineari evidenti.

In [144]:
data.corr(numeric_only=True)

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,IS_BAD_PAYER,CODE_GENDER_F,CODE_GENDER_M
ID,1.000000,-0.011163,-0.098851,0.028878,-0.017667,0.009211,0.056016,0.005745,0.026624,0.004892,-0.012022,0.012022
FLAG_OWN_CAR,-0.011163,1.000000,-0.015185,0.105839,0.215506,0.101272,0.157144,-0.006244,0.151814,0.001992,-0.361379,0.361379
FLAG_OWN_REALTY,-0.098851,-0.015185,1.000000,-0.000575,0.032719,-0.010997,-0.129838,0.033646,-0.005723,-0.010987,0.050758,-0.050758
CNT_CHILDREN,0.028878,0.105839,-0.000575,1.000000,0.033691,0.049823,0.339357,-0.043358,0.889114,-0.003710,-0.077690,0.077690
AMT_INCOME_TOTAL,-0.017667,0.215506,0.032719,0.033691,1.000000,0.226931,0.067908,-0.087130,0.023750,0.007093,-0.197805,0.197805
NAME_EDUCATION_TYPE,0.009211,0.101272,-0.010997,0.049823,0.226931,1.000000,0.169024,-0.016347,0.041344,0.010081,0.005880,-0.005880
DAYS_BIRTH,0.056016,0.157144,-0.129838,0.339357,0.067908,0.169024,1.000000,-0.023497,0.304020,-0.006954,-0.202352,0.202352
DAYS_EMPLOYED,0.005745,-0.006244,0.033646,-0.043358,-0.087130,-0.016347,-0.023497,1.000000,-0.054587,0.018482,-0.031731,0.031731
CNT_FAM_MEMBERS,0.026624,0.151814,-0.005723,0.889114,0.023750,0.041344,0.304020,-0.054587,1.000000,-0.005718,-0.110782,0.110782
IS_BAD_PAYER,0.004892,0.001992,-0.010987,-0.003710,0.007093,0.010081,-0.006954,0.018482,-0.005718,1.000000,-0.014632,0.014632


Come si può intuire, le features "CNT_FAM_MEMBERS" e "CNT_CHILDREN" sono altamente correlate, quindi possiamo scartare una delle due.

In [145]:
data = data.drop(columns=['CNT_CHILDREN'])

A questo punto restano quattro features di cui fare l'encoding. \
Si può pensare di fare una one-hot encoding per ogni variabile ma questo processo andrebbe ad aumentare di molto il numero delle features del dataset in modo forse non propriamente utile, perciò si può provare una frequency encoding, ovvero si può codificare ogni categoria di ogni variabile categorica con la frequenza del valore positivo della variabile target. Questo dovrebbe evidenziare (qualora ci fosse) la correlazione tra le features considerate e la variabile target. \
Con questo approccio si rischia però data leakage, ovvero se si fa la frequency encoding su tutto il dataset si darebbe al modello l'informazione sulla frequenza delle osservazioni del test anche in fase di training. \
Perciò si deve dividere il modello in train e test per completare questa fase di preprocessing.

In [146]:
def freq_encoding (X_train, X_test, Y_train, Y_test):

    train, test = train_test_split ( data ,test_size=.3 , random_state=10)
    
    freq_occ = train.groupby('OCCUPATION_TYPE')['IS_BAD_PAYER'].sum()
    freq_name = train.groupby('NAME_INCOME_TYPE')['IS_BAD_PAYER'].sum()
    freq_fam = train.groupby('NAME_FAMILY_STATUS')['IS_BAD_PAYER'].sum()
    freq_house = train.groupby('NAME_HOUSING_TYPE')['IS_BAD_PAYER'].sum()
    
    X_train = train.drop("IS_BAD_PAYER", axis=1)
    Y_train = train["IS_BAD_PAYER"]
    
    X_test = test.drop("IS_BAD_PAYER", axis=1)
    Y_test = test["IS_BAD_PAYER"]
    
    X_train['OCCUPATION_TYPE'] = X_train['OCCUPATION_TYPE'].map(freq_occ)
    X_train['NAME_INCOME_TYPE'] = X_train['NAME_INCOME_TYPE'].map(freq_name)
    X_train['NAME_FAMILY_STATUS'] = X_train['NAME_FAMILY_STATUS'].map(freq_fam)
    X_train['NAME_HOUSING_TYPE'] = X_train['NAME_HOUSING_TYPE'].map(freq_house)
    
    X_test['OCCUPATION_TYPE'] = X_test['OCCUPATION_TYPE'].map(freq_occ)
    X_test['NAME_INCOME_TYPE'] = X_test['NAME_INCOME_TYPE'].map(freq_name)
    X_test['NAME_FAMILY_STATUS'] = X_test['NAME_FAMILY_STATUS'].map(freq_fam)
    X_test['NAME_HOUSING_TYPE'] = X_test['NAME_HOUSING_TYPE'].map(freq_house)
    
    return X_train, X_test, Y_train, Y_test

Definiamo inoltre due funzioni di valutazione del modello, la funzione "evaluate_model_proba" in particolare sarà utile per regolare la soglia di probabilità di appartenenza alla classe positiva.

In [147]:
def evaluate_model(model, dataset, subset):
    X, y_true = dataset
    y_pred = model.predict(X)
    print(f"METRICHE SUL {subset}:\n ")
    print(f"Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"Recall: {recall_score(y_true, y_pred):.3f}")
    print(f"f1 score: {f1_score(y_true, y_pred):.3f}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.3f} \n")
    print(f"MATRICE DI CONFUSIONE: \n {confusion_matrix(y_true, y_pred)}") 

In [148]:
def evaluate_model_proba(model, dataset, subset, threshold):
    X, y_true = dataset
    y_proba_pred = model.predict_proba(X)[:,1]
    
    binary_predictions = (y_proba_pred > threshold).astype(int)

    print(f"METRICHE SUL {subset}:\n ")
    print(f"Precision: {precision_score(y_true, binary_predictions):.3f}")
    print(f"Recall: {recall_score(y_true, binary_predictions):.3f}")
    print(f"f1 score: {f1_score(y_true, binary_predictions):.3f}")
    print(f"Accuracy: {accuracy_score(y_true, binary_predictions):.3f} \n")
    print(f"MATRICE DI CONFUSIONE: \n {confusion_matrix(y_true, binary_predictions)}") 

# Selezione del modello

Nella scelta del modello bisogna tenere presente che un punto chiave di questo problema è l'interpretabilità del risultato, cioè il modello deve anche riuscire a dare indicazioni su quali features abbiano pesato nella scelta della classe. \
Per questo motivo, scegliamo di non usare due metodi molto potenti come k-nn e reti neurali poichè l'interpretazione di questi modelli è molto difficile se non impossibile.

Possiamo provare a vedere se è il caso di usare support vector machines, per questo andiamo a verificare se c'è una significativa presenza di outlier nel dataset.

In [149]:
outlier_check = ['AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS']
outliers = []

for name_col in outlier_check:
    Q1 = data[f'{name_col}'].quantile(0.25)
    Q3 = data[f'{name_col}'].quantile(0.75)
    IQR = Q3 - Q1

    outliers = data[(data[f'{name_col}'] < (Q1 - 1.5 * IQR)) | (data[f'{name_col}'] > (Q3 + 1.5 * IQR))]
    
    print (f'Colonna: {name_col}, numero di outliers: {outliers.shape[0]}')


Colonna: AMT_INCOME_TOTAL, numero di outliers: 1529
Colonna: DAYS_BIRTH, numero di outliers: 0
Colonna: DAYS_EMPLOYED, numero di outliers: 1770
Colonna: CNT_FAM_MEMBERS, numero di outliers: 480


In questo caso nel dataset ci sono molte osservazioni, non c'è una significativa presenza di outlier e quindi nemmeno i modelli support vector machines sembrano prestarsi bene a questo problema. Inoltre, i modelli svm non sono di facile interpretazione quando il kernel è diverso da quello lineare.\
Dunque, proviamo due modelli che hanno una buona interpretabilità: Logistic Regression e Naive Bayes. Infatti, con i coefficienti della regressione logistica, si può vedere quali features hanno pesato di più nella classificazione, mentre con Naive Bayes si può calcolare la probabilità di apparteneza ad una classe per una determinata osservazione e vedere quali sono le  features che hanno pesato di più come vedremo più avanti.

## Logistic Regression

A priori, non ci si aspetta grandi risutati dalla regressione logistica dal momento che, come visto nella parte di preprocessing non c'è troppa correlazione lineare tra le variabili.

In [150]:
Y = data["IS_BAD_PAYER"].values
X = data.drop("IS_BAD_PAYER", axis=1).values

X_train, X_test, Y_train, Y_test = train_test_split ( X,Y,test_size=.3 , random_state=10 )

X_train, X_test, Y_train, Y_test = freq_encoding(X_train, X_test, Y_train, Y_test)

ss = StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)

lr = LogisticRegression(class_weight="balanced")
lr.fit(X_train, Y_train)

evaluate_model(lr, (X_train, Y_train) , "TRAIN")

evaluate_model(lr, (X_test, Y_test) , "TEST")

METRICHE SUL TRAIN:
 
Precision: 0.008
Recall: 0.632
f1 score: 0.015
Accuracy: 0.579 

MATRICE DI CONFUSIONE: 
 [[14687 10699]
 [   49    84]]
METRICHE SUL TEST:
 
Precision: 0.008
Recall: 0.571
f1 score: 0.015
Accuracy: 0.569 

MATRICE DI CONFUSIONE: 
 [[6189 4686]
 [  27   36]]


Come ci si poteva aspettare, la regressione logistica non è un buon modello predittivo in questo caso, scartiamo dunque questa strada.

## Naive Bayes

Per la scelta della probabilità a priori dell'algoritmo abbiamo due strade: si può scegliere di utilizzare la distribuzione Gaussiana in quanto le features "AMT_INCOME_ANNUAL", "DAYS_BIRTH" e "DAYS_EMPLOYED" sono continue oppure, visto che nella parte di preprocessing abbiamo scelto la frequency encoding, possiamo provare a trasformare le variabili continue in conteggi e usare quindi una distribuzione multinomiale oppure Complement Naive Bayes che dovrebbe essere più adatto a gestire lo sbilanciamento delle classi. Probabilmente con quest'ultimo approccio però si possono perdere informazioni sulle variabili continue.

Dal momento che Naive Bayes assume l'indipendenza tra le features, modifichiamo le colonne ottenute con la one hot encoding per la variabile indicante il sesso. Infatti le due features "CODE_GENDER_F" e "CODE_GENDER_M" saranno chiaramente dipendenti. In questo caso ci basta semplicemente rimuovere una delle due colonne.

In [151]:
data=data.drop(columns=['CODE_GENDER_F'])

In più si può intuire che fissando due valori per "DAYS_BIRTH" e "DAYS_EMPLOYED" la colonna "AMT_INCOME_TOTAL" assume un unico valore. Proviamo a vedere se questa intuizione è fondata.

In [152]:
data['combinazione'] = data['DAYS_BIRTH'].astype(str) + '_' + data['DAYS_EMPLOYED'].astype(str)

count_amt = data.groupby('combinazione')['AMT_INCOME_TOTAL'].nunique()

ones = count_amt[count_amt!=1]

ones.shape[0]

182

Per sole 182 osservazioni su più di 35000 del dataset si ha un valore non unico di "AMT_INCOME_TOTAL" una volta fissato "DAYS_BIRTH" e "DAYS_EMPLOYED". Perciò queste tre features non possono essere indipendenti.

Proviamo a scartare la variabile "AMT_INCOME_TOTAL" dal dataset.

In [153]:
data = data.drop(columns=['AMT_INCOME_TOTAL', 'combinazione'])

### Gaussian Naive Bayes

Provo con l'undersampling a gestire lo sbilanciamento delle classi e fisso una soglia di 0.3 (che eventualmente è un parametro da migliorare), per aumentare la recall.

In [154]:
Y = data["IS_BAD_PAYER"].values
X = data.drop("IS_BAD_PAYER", axis=1).values

X_train, X_test, Y_train, Y_test = train_test_split ( X,Y,test_size=.3 , random_state=10 )

X_train, X_test, Y_train, Y_test = freq_encoding(X_train, X_test, Y_train, Y_test)

undersampler = RandomUnderSampler(sampling_strategy='auto' )

X_train, Y_train = undersampler.fit_resample(X_train, Y_train)

gnb = GaussianNB()
gnb.fit(X_train, Y_train)

evaluate_model_proba(gnb, (X_train,Y_train), "TRAIN", 0.3)
evaluate_model_proba(gnb, (X_test,Y_test), "TEST", 0.3)

METRICHE SUL TRAIN:
 
Precision: 0.521
Recall: 0.925
f1 score: 0.667
Accuracy: 0.538 

MATRICE DI CONFUSIONE: 
 [[ 20 113]
 [ 10 123]]
METRICHE SUL TEST:
 
Precision: 0.006
Recall: 0.921
f1 score: 0.012
Accuracy: 0.158 

MATRICE DI CONFUSIONE: 
 [[1675 9200]
 [   5   58]]


Proviamo a vedere se questo modello soffre di overfitting.

In [155]:
kf = KFold(n_splits=5, shuffle =True)
i=1
X = data.drop("IS_BAD_PAYER", axis=1).values

for train_index , test_index in kf.split(X):
    
    X_train , X_test = X[train_index] , X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    X_train, X_test, Y_train, Y_test = freq_encoding(X_train, X_test, Y_train, Y_test)

    undersampler = RandomUnderSampler(sampling_strategy='auto')
    X_train, Y_train = undersampler.fit_resample(X_train, Y_train)
    
    gnb = GaussianNB()
    gnb.fit(X_train, Y_train)
    
    print(f"Metriche dopo {i} split di cross-validation: \n")
    
    evaluate_model_proba(gnb, (X_train,Y_train), "TRAIN", 0.3)
    evaluate_model_proba(gnb, (X_test,Y_test), "TEST", 0.3)

    i+=1

Metriche dopo 1 split di cross-validation: 

METRICHE SUL TRAIN:
 
Precision: 0.519
Recall: 0.932
f1 score: 0.667
Accuracy: 0.534 

MATRICE DI CONFUSIONE: 
 [[ 18 115]
 [  9 124]]
METRICHE SUL TEST:
 
Precision: 0.006
Recall: 0.905
f1 score: 0.012
Accuracy: 0.146 

MATRICE DI CONFUSIONE: 
 [[1538 9337]
 [   6   57]]
Metriche dopo 2 split di cross-validation: 

METRICHE SUL TRAIN:
 
Precision: 0.534
Recall: 0.940
f1 score: 0.681
Accuracy: 0.560 

MATRICE DI CONFUSIONE: 
 [[ 24 109]
 [  8 125]]
METRICHE SUL TEST:
 
Precision: 0.006
Recall: 0.921
f1 score: 0.012
Accuracy: 0.144 

MATRICE DI CONFUSIONE: 
 [[1521 9354]
 [   5   58]]
Metriche dopo 3 split di cross-validation: 

METRICHE SUL TRAIN:
 
Precision: 0.515
Recall: 0.925
f1 score: 0.661
Accuracy: 0.526 

MATRICE DI CONFUSIONE: 
 [[ 17 116]
 [ 10 123]]
METRICHE SUL TEST:
 
Precision: 0.007
Recall: 0.952
f1 score: 0.013
Accuracy: 0.163 

MATRICE DI CONFUSIONE: 
 [[1719 9156]
 [   3   60]]
Metriche dopo 4 split di cross-validation: 

M

Il modello non sembra soffrire di overfitting.

### Interpretabilità

Tenendo presente che se y_pred è la classe predetta da Naive Bayes allora

<center>
    y_pred = argmax(P(y) * ∏ P(x_i | y))
</center>

e che P(y) è uguale per entrambe le classi visto che stiamo bilanciando il dataset con l'undersampling e che 

<center>
    P(x_i | y) = (1 / (σ_y * sqrt(2 * π))) * exp(-(x_i - μ_y)^2 / (2 * σ_y^2))
</center>

si può osservare che più x_i è vicino alla media della feature i sulla classe y (μ_y) più l'esponenziale sarà vicino a 1 e quindi il valore P(x_i | y) sarà vicino al valore massimo.\
Perciò, supponiamo che l'osservazione x sia stata classificata come appartenente alla classe 1 cioè ∏ P(x_i | y=1) > ∏ P(x_i | y=0) (dimentichiamoci per un attimo del fatto che c'è una threshold). Fissato un i, se P(x_i | y=1) >> P(x_i | y=0) allora la feature i ha avuto un peso significativo nella scelta della classe.

La seguente funzione "probabilites", data una osservazione x, individua per ogni i la differenza tra P(x_i | y=1) e P(x_i | y=0) in valore assoluto e restituisce tutte le features in ordine decrescente rispetto alla differenza in valore assoluto tra P(x_i | y=1) e P(x_i | y=0). Quindi le prime features dell'output saranno quelle più influenti nella scelta della classe.

In [156]:
def probabilities (x):
    positive_class = []
    negative_class = []
    
    prob_pos = 1
    prob_neg = 1
    
    difference = []
    difference_norm = []
    
    list_index = []
    
    for feature_index in range(X_train.shape[1]):
        mean_neg = gnb.theta_[0, feature_index]
        mean_pos = gnb.theta_[1, feature_index]
    
        std_dev_neg = np.sqrt(gnb.var_[0, feature_index])
        std_dev_pos = np.sqrt(gnb.var_[1, feature_index])
        
        conditional_prob_pos = (1 / (np.sqrt(2 * np.pi) * std_dev_pos)) * np.exp(-(x[feature_index] - mean_pos) ** 2 / (2 * std_dev_pos ** 2))
        conditional_prob_neg = (1 / (np.sqrt(2 * np.pi) * std_dev_neg)) * np.exp(-(x[feature_index] - mean_neg) ** 2 / (2 * std_dev_neg ** 2))
        
        
        positive_class.append(conditional_prob_pos)
        negative_class.append(conditional_prob_neg)
        
    for val in positive_class:
        prob_pos*=val
    
    for val_n in negative_class:
        prob_neg*=val_n
        
    norm_pos = prob_pos / (prob_pos+prob_neg)
    norm_neg = prob_neg / (prob_pos+prob_neg)
    
    pred_class = (1 if norm_pos > norm_neg or norm_pos >= 0.3 else 0)  #threshold=0.3
    
    print(f"Probabilità che sia un cattivo pagatore: {norm_pos:.2f}")
    print(f"Probabilità che sia un buon pagatore: {norm_neg:.2f}\n")
    print(f"Classe predetta: {pred_class} \n")
    for elem1, elem2 in zip(positive_class, negative_class):
        difference.append(abs(elem1 - elem2))
    
    for i in range(len(difference)):
        
        index = difference.index(max(difference))
        list_index.append(index)
        difference[index] = 0
    
    print("Features ordinate per importanza nella scelta della classe: \n ")
    
    for j in list_index:
        print(f"{X_train.columns[j]} ")
 

In [157]:
gnb.predict_proba(X_train)[:,1]>0.3

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

La prima osservazione del train ad esempio è stata classificata come cattivo pagatore, vediamo quali sono state le features che hanno influito di più nella scelta.

In [158]:
probabilities(X_train.iloc[0].to_list())

Probabilità che sia un cattivo pagatore: 0.45
Probabilità che sia un buon pagatore: 0.55

Classe predetta: 1 

Features ordinate per importanza nella scelta della classe: 
 
NAME_EDUCATION_TYPE 
CODE_GENDER_M 
FLAG_OWN_REALTY 
CNT_FAM_MEMBERS 
NAME_FAMILY_STATUS 
NAME_INCOME_TYPE 
OCCUPATION_TYPE 
NAME_HOUSING_TYPE 
DAYS_EMPLOYED 
DAYS_BIRTH 
ID 
ID 


### Multinomial Naive Bayes

Proviamo ora ad utilizzare una probabilità a priori diversa da quella Gaussiana. Per poter usare Multinomial Naive Bayes oppure Complement Naive Bayes che in teoria è anche adatto a gestire lo sbilanciamento delle classi, bisogna fare qualche modifica alle variabili continue del dataset per renderle discrete.

Proviamo a dividere le features continue utilizzando i quantili e quindi dividendole in quattro intervalli.

In [159]:
def disc_features(col):
    
    quantili = data[f'{col}'].quantile([0, 0.25, 0.5, 0.75, 1])

    intervalli = [float('-inf'), quantili[0.25], quantili[0.5], quantili[0.75], float('inf')]

    etichette = [1, 2, 3, 4]


    data[f'{col}'] = pd.cut(data[f'{col}'], bins=intervalli, labels=etichette)    

In [160]:
#disc_features('AMT_INCOME_TOTAL')
disc_features('DAYS_BIRTH')
disc_features('DAYS_EMPLOYED')

In [161]:
Y = data["IS_BAD_PAYER"].values
X = data.drop("IS_BAD_PAYER", axis=1).values

X_train, X_test, Y_train, Y_test = train_test_split ( X,Y,test_size=.3 , random_state=10 )

X_train, X_test, Y_train, Y_test = freq_encoding(X_train, X_test, Y_train, Y_test)

undersampler = RandomUnderSampler(sampling_strategy='auto')
X_train, Y_train = undersampler.fit_resample(X_train, Y_train)


mnb = MultinomialNB()
mnb.fit(X_train, Y_train)

evaluate_model_proba(mnb, (X_train,Y_train), "TRAIN", 0.3)
evaluate_model_proba(mnb, (X_test,Y_test), "TEST", 0.3)

METRICHE SUL TRAIN:
 
Precision: 0.528
Recall: 0.714
f1 score: 0.607
Accuracy: 0.538 

MATRICE DI CONFUSIONE: 
 [[48 85]
 [38 95]]
METRICHE SUL TEST:
 
Precision: 0.006
Recall: 0.683
f1 score: 0.011
Accuracy: 0.309 

MATRICE DI CONFUSIONE: 
 [[3333 7542]
 [  20   43]]


In [162]:
Y = data["IS_BAD_PAYER"].values
X = data.drop("IS_BAD_PAYER", axis=1).values

X_train, X_test, Y_train, Y_test = train_test_split ( X,Y,test_size=.3 , random_state=10 )

X_train, X_test, Y_train, Y_test = freq_encoding(X_train, X_test, Y_train, Y_test)

smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)


mnb = MultinomialNB()
mnb.fit(X_train, Y_train)

evaluate_model_proba(mnb, (X_train,Y_train), "TRAIN", 0.3)
evaluate_model_proba(mnb, (X_test,Y_test), "TEST", 0.3)

METRICHE SUL TRAIN:
 
Precision: 0.594
Recall: 0.780
f1 score: 0.674
Accuracy: 0.623 

MATRICE DI CONFUSIONE: 
 [[11832 13554]
 [ 5590 19796]]
METRICHE SUL TEST:
 
Precision: 0.006
Recall: 0.571
f1 score: 0.012
Accuracy: 0.475 

MATRICE DI CONFUSIONE: 
 [[5162 5713]
 [  27   36]]


L'approccio scelto con Multinomial Naive Bayes non produce un buon modello, nè con oversampliing nè con undersampling, questo perchè probabilmente nella trasformazione delle variabili continue in discrete sono state perse informazioni preziose per la classificazione. 

## Complement Naive Bayes

In [163]:
Y = data["IS_BAD_PAYER"].values
X = data.drop("IS_BAD_PAYER", axis=1).values

X_train, X_test, Y_train, Y_test = train_test_split ( X,Y,test_size=.3 , random_state=10 )

X_train, X_test, Y_train, Y_test = freq_encoding(X_train, X_test, Y_train, Y_test)

cnb = ComplementNB()
cnb.fit(X_train, Y_train)

evaluate_model_proba(cnb, (X_train,Y_train), "TRAIN", 0.3)
evaluate_model_proba(cnb, (X_test,Y_test), "TEST", 0.3)

METRICHE SUL TRAIN:
 
Precision: 0.006
Recall: 0.812
f1 score: 0.012
Accuracy: 0.308 

MATRICE DI CONFUSIONE: 
 [[ 7743 17643]
 [   25   108]]
METRICHE SUL TEST:
 
Precision: 0.007
Recall: 0.794
f1 score: 0.013
Accuracy: 0.318 

MATRICE DI CONFUSIONE: 
 [[3429 7446]
 [  13   50]]


Complement Naive Bayes produce un modello che ha una buona recall anche senza bilanciare le classi ma comunque funziona peggio rispetto a Gaussian NB.

## Conclusioni 

In definitiva, il modello Gaussian Naive Bayes sembra essere il modello più adatto al caso di business tra quelli considerati perchè produce una buona recall e ha una buona interpretabilità.